In [ ]:
# Kiểm tra setup
try:
    from shared_variables import *
except:
    # Tải các biến và hàm từ notebook 1
    %run "1_Setup_and_Utils.ipynb"

# Các hàm xử lý ảnh
def resize_image(image, target_size, preserve_aspect_ratio=True):
    """
    Resize an image to the target size with aspect ratio preserved
    """
    target_w, target_h = target_size
    
    if isinstance(image, np.ndarray):
        # OpenCV image
        if preserve_aspect_ratio:
            h0, w0 = image.shape[:2]
            scale = min(target_w / w0, target_h / h0)
            new_w, new_h = int(w0 * scale), int(h0 * scale)
            
            # Resize theo tỉ lệ
            resized = cv2.resize(image, (new_w, new_h))
            
            # Tạo ảnh nền đen
            if len(image.shape) == 3:
                result = np.zeros((target_h, target_w, image.shape[2]), dtype=image.dtype)
            else:
                result = np.zeros((target_h, target_w), dtype=image.dtype)
            
            # Tính toán padding để căn giữa
            pad_w = target_w - new_w
            pad_h = target_h - new_h
            pad_left = pad_w // 2
            pad_top = pad_h // 2
            
            # Đặt ảnh resize vào giữa
            if len(image.shape) == 3:
                result[pad_top:pad_top+new_h, pad_left:pad_left+new_w, :] = resized
            else:
                result[pad_top:pad_top+new_h, pad_left:pad_left+new_w] = resized
                
            return result
        else:
            return cv2.resize(image, target_size)
    else:
        # PIL Image
        if preserve_aspect_ratio:
            w0, h0 = image.size
            scale = min(target_w / w0, target_h / h0)
            new_w, new_h = int(w0 * scale), int(h0 * scale)
            
            resized = image.resize((new_w, new_h), Image.LANCZOS)
            
            if image.mode == "L":
                result = Image.new("L", (target_w, target_h), 0)
            else:
                result = Image.new("RGB", (target_w, target_h), (0, 0, 0))
                
            x_offset = (target_w - new_w) // 2
            y_offset = (target_h - new_h) // 2
            
            result.paste(resized, (x_offset, y_offset))
            return result
        else:
            return image.resize(target_size, Image.LANCZOS)

def convert_to_grayscale(image):
    """
    Convert an image to grayscale
    """
    if isinstance(image, np.ndarray):
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        return image.convert("L")

def extract_hand_region(image, roi_size=300):
    """
    Extract the hand region from the image using skin color detection
    """
    # Define region of interest (ROI) in the center-right of the frame
    height, width = image.shape[:2]
    roi_x = width // 2 - roi_size // 2
    roi_y = height // 2 - roi_size // 2
    
    # Ensure ROI is within image bounds
    roi_x = max(0, min(roi_x, width - roi_size))
    roi_y = max(0, min(roi_y, height - roi_size))
    
    # Extract ROI
    roi = image[roi_y:roi_y+roi_size, roi_x:roi_x+roi_size].copy()
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define range for skin color detection
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    # Create binary mask for skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # Apply morphological operations to improve the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=2)
    mask = cv2.erode(mask, kernel, iterations=1)
    
    # Apply Gaussian blur to reduce noise
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw ROI boundary for visual guidance
    cv2.rectangle(image, (roi_x, roi_y), (roi_x + roi_size, roi_y + roi_size), 
                 (255, 0, 0), 2)
    
    if contours:
        # Find the largest contour (assume it's the hand)
        max_contour = max(contours, key=cv2.contourArea)
        contour_area = cv2.contourArea(max_contour)
        
        # If the contour is large enough (avoid small noise)
        if contour_area > 3000:
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(max_contour)
            
            # Add some padding
            padding = 20
            x = max(0, x - padding)
            y = max(0, y - padding)
            w = min(roi_size - x, w + 2*padding)
            h = min(roi_size - y, h + 2*padding)
            
            # Extract the hand region from the ROI
            hand = roi[y:y+h, x:x+w]
            
            # Draw the rectangle on original image for visualization
            cv2.rectangle(image, (roi_x + x, roi_y + y), 
                         (roi_x + x + w, roi_y + y + h), (0, 255, 0), 2)
            
            # Display contour area for debugging
            cv2.putText(image, f"Area: {contour_area:.0f}", (roi_x, roi_y - 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            if hand.size > 0:
                # Thêm text khi phát hiện tay
                cv2.putText(image, "Hand Detected", (roi_x, roi_y - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                return hand
    
    # If no good hand region is detected
    cv2.putText(image, "No hand detected", (roi_x, roi_y - 10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
               
    # Return a blurred version of ROI instead of the original one
    blurred_roi = cv2.GaussianBlur(roi, (25, 25), 0)
    return blurred_roi

def draw_prediction(image, sign, confidence):
    """
    Draw prediction text on the image
    """
    # Create a copy of the image
    result = image.copy()
    
    # Draw a semi-transparent rectangle for text background
    overlay = result.copy()
    cv2.rectangle(overlay, (10, 10), (300, 140), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.6, result, 0.4, 0, result)
    
    # Define text to display
    if sign == "?" or confidence < 0.5:
        text = "Waiting for hand gesture..."
        color = (0, 0, 255)  # Red
    else:
        text = f"Sign: {sign} ({confidence:.2f})"
        color = (0, 255, 0)  # Green
    
    # Draw the text
    cv2.putText(result, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
    # Draw hand placement guide
    height, width = image.shape[:2]
    roi_size = 300
    roi_x = width // 2 - roi_size // 2
    roi_y = height // 2 - roi_size // 2
    cv2.rectangle(result, (roi_x, roi_y), (roi_x + roi_size, roi_y + roi_size), 
                 (255, 0, 0), 2)
    
    return result

# Lưu các hàm xử lý ảnh để chia sẻ với các notebook khác
save_variable(resize_image, 'resize_image')
save_variable(convert_to_grayscale, 'convert_to_grayscale')
save_variable(extract_hand_region, 'extract_hand_region')
save_variable(draw_prediction, 'draw_prediction')

# Test các hàm với ảnh mẫu (tùy chọn)
print("Bạn có muốn thử nghiệm các hàm xử lý ảnh với một ảnh mẫu? (y/n)")
test_image = input()
if test_image.lower() == 'y':
    print("Vui lòng upload một ảnh để thử nghiệm:")
    uploaded = files.upload()
    
    if uploaded:
        for filename, content in uploaded.items():
            image = cv2.imdecode(np.frombuffer(content, np.uint8), cv2.IMREAD_COLOR)
            
            print("Ảnh gốc:")
            display_image(image)
            
            print("Ảnh được resize về 64x64:")
            resized = resize_image(image, (64, 64))
            display_image(resized)
            
            print("Ảnh grayscale:")
            gray = convert_to_grayscale(image)
            display_image(gray)
            
            print("Phát hiện vùng tay:")
            hand_region = extract_hand_region(image.copy())
            display_image(image)  # Show image with bounding box
            display_image(hand_region)  # Show extracted hand region